In [117]:
"""
les symboles particulières comme ? ! et . sont supprimé dans la question.
Il faut peut être returner une réponse spécifique si il rencontre plus 1 mot qu'il existe pas dans le corpus
Nous suprimmons également:
  1. Mot qui contient une seule lettre(qui sont suivante des erreurs)
  2. Une fonction unique pour nettoyage tous les données dans train-test et chat
Nous allons traiter tous les mots clés suivant :
km, vitesse, carburant, huile, position, batterie, pression_pneu. Il reste trajet et vin
  
"""
#import sys
#!{sys.executable} -m pip uninstall gtts

#from gtts import gTTS
#from playsound import playsound
#import speech_recognition
import time
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('french')
from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords = False)

import csv
import re
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import math
import time
from dateparser.search import search_dates
import pandas as pd


def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word


def clear_line(pharse):
    """
    Arg: string
    Return: string
    """
    pharse_ =[]
    for words in pharse.strip().strip('?').strip('.').strip('!').lower().split(' '):
        for word in words.split('-'):
            for word_ in word.split('\''):
                word_ = clearing_word(word_)
                word_ = stemmer.stem(word_)
        if len(word_) >1:       
            pharse_.append(word_)  
    return ' '.join(pharse_)

def tf_idf(word, doc):
    tf, df = 0, 0
    if word in str(doc):
        tf = 1
    for doc_ex in QUESTION_TO_TRAINS:
        if word in str(doc_ex):
            df+=1
    if df>0:
            return tf*math.log(len(QUESTION_TO_TRAINS)/(df), 10)
    else:
            return 0
    
def split_words(doc):
    
    list1 = doc.split()
    list2 = []
    for i in range(len(list1)-1):
        word = list1[i]+ ' '+list1[i+1]
        list2.append(word)
    for word in list1:
        list2.append(word)
    return list2
    
    
def dict_doc_score(doc): 
    list_word = split_words(doc)
    return {word: tf_idf(word, doc) for word in list_word}
   
    
def score_by_new_doc(new_doc, doc, DICT):
    
    score = 0
    for word in new_doc.split():
        if word in DICT[doc]:
                score += DICT[doc][word]
    return score

def score_by_new_list(list_, doc, DICT):
    
    score = 0
    for word in list_:
        if word in DICT[doc]:
                score += DICT[doc][word]
    return score    

        

def find_right_index(new_doc, DICT):
    
    #index_train = random.sample(range(len(QUESTION_TO_TRAINS)),int(len(QUESTION_TO_TRAINS)*0.25))
    index_train = range(len(QUESTION_TO_TRAINS))
    train = [QUESTION_TO_TRAINS[i] for i in index_train]
    index=0
    biggest_score = score_by_new_doc(new_doc, train[0], DICT)
    for i in range(len(train)):
        boolean1 = score_by_new_doc(new_doc, train[i], DICT) > biggest_score
        boolean2 = score_by_new_doc(new_doc, train[i], DICT) == biggest_score
        boolean3 = len(train[index]) > len(train[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_doc(new_doc, train[i], DICT)
            index = i
    index = index_train[index]
    return index

def test(DICT, méthode, train =False):
    
    if not train:
        index_to_print = random.sample(range(len(pairs_tests)), 20)
        #loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            #loss_total +=loss
            if i in index_to_print:
                print(pairs_tests[i])
                print(pairs_trains[index][1])
        print('-'*80)
       # print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        #loss_total = 0
        #index_to_print = random.sample(range(len(pairs_trains)), 20)  
        list_of_bad_prediction = []
        i0 = 0
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            #loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            #loss_total +=loss
            if index != i :
                i0 +=1
                print(pairs_trains[i])
                print(pairs_trains[index][1])
        print('-'*80)
        #print('ACCURACY=', 1-loss_total/len(pairs_trains))
        print('Il y a {} erreurs d\'indexes parmis {} prédictions'.format(i0, len(pairs_trains)))

def test_without_print(DICT, méthode, train =False):
    
    if not train:
        loss_total = 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_tests[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_tests))
    if  train:
        loss_total = 0
        index_to_print = random.sample(range(len(pairs_trains)), 20)   
        for i in range(len(pairs_trains)):
            index = find_right_index(pairs_trains[i][0], DICT, méthode)
            loss = _evaluate_by_right_word(pairs_trains[index][1], pairs_trains[i][1])
            loss_total +=loss
        print('-'*80)
        print('ACCURACY=', 1-loss_total/len(pairs_trains))
        
        
def chat():
    
    print('Bonjour, C\'est le bot d\'Avicen, pose tes questions, s\'il te plaît!')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    while True:
            line = str(input('Vous: '))
            line = line.lower().strip().strip('?').strip('.')
            line = clear_line(line)
            if len(line) !=0:
                index= find_right_index(line, DICT)
                print('bot: ', pairs_trains[index][1])
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')
            
            else:
                f.close()
                break 
 
def find_right_index_in_list(list_):
    train = QUESTION_TO_TRAINS
    index=0
    biggest_score = score_by_new_list(list_, train[0], DICT)
    for i in range(len(train)):
        boolean1 = score_by_new_list(list_, train[i], DICT) > biggest_score
        boolean2 = score_by_new_list(list_, train[i], DICT) == biggest_score
        boolean3 = len(train[index]) > len(train[i])
        if boolean1 or (boolean2 and boolean3):
            biggest_score = score_by_new_list(list_, train[i], DICT)
            index = i
    return index
    

def get_key_and_find_right_index(dict_):
    """
    we suppose that length of dict is bigger than 4"""
    sorted_ = sorted(dict_.items(), key=lambda kv: kv[1])
    list_ = sorted_[-4:]
    list_of_key = []
    for l in list_:
        list_of_key.append(l[0])
    index = find_right_index_in_list(list_of_key)
    return index


def get_important_term(line):
    if len(line) <2:
        return None, None
    list_ = split_words(line)
    word = list_[0]
    score = 0
    for doc in QUESTION_TO_TRAINS:
        for term in list_:
            if term in DICT[doc]:
                if DICT[doc][term] > score:
                    word = term
                    score = DICT[doc][term]
    return word, score
                
                        
def chat_with_memory():
    
    print('Bonjour, C\'est le bot d\'Avicen, comment ça va?')
    path = 'processed/chat_tfidf.txt'
    f =  open(path, 'a+') 
    attentions = {}
    while True:
            line = str(input('Vous: '))
            line = line.lower().strip().strip('?').strip('.')
            line = clear_line(line)
            term, score = get_important_term(line)
            if len(attentions) >10:
                for i in range(3):
                    keys = list(attentions.keys())
                    key  = random.choice(keys)
                    del attentions[key]
            if term != None:
                attentions[term] = score
            if len(line) > 1:
                index = find_right_index(line, DICT)
                print('bot: ', pairs_trains[index][1])
                random_ = random.choice(range(10))
                if len(attentions) >4 and random_ >6:    
                    index = get_key_and_find_right_index(attentions)
                    print('bot: ', raw_questions[index])
                f.write('VOUS ++++ '+line+'\n')
                f.write('BOT ++++ '+pairs_trains[index][1]+'\n')         
            else:
                f.close()
                break 

def precision_recall(lstcomp, lstref):
    
    card_intersec = 0.0 # force à utiliser la division non entière
    for t in set(lstcomp) :
        card_intersec += min(lstref.count(t), lstcomp.count(t))
    if len(lstcomp)==0:
        precision =1
    else:
        precision = card_intersec/len(lstcomp)
    if len(lstref)==0:
        rappel = 1
    else:
        rappel = card_intersec/len(lstref)
    return (precision, rappel)


def test_with_precision_recall(DICT,  train = False):
    
    if not train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_tests)):
            index = find_right_index(pairs_tests[i][0], DICT)
            precision, recall = precision_recall\
            (pairs_tests[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_tests)
        total_recall = total_recall/len(pairs_tests)      
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall,F))
    if  train:
        total_precision, total_recall = 0, 0
        for i in range(len(pairs_trains)):          
            index = find_right_index(pairs_trains[i][0], DICT)
            precision, recall = precision_recall\
            (pairs_trains[i][1], pairs_trains[index][1])
            total_precision  += precision
            total_recall     += recall
        print('-'*80)
        total_precision = total_precision/len(pairs_trains)
        total_recall = total_recall/len(pairs_trains)
        F = 2*total_precision*total_recall/(total_precision+total_recall)
        print('PRECISION = {:.5f},  RECALL = {:.5f}, F_MESURE = {:.3f} '\
              .format(total_precision, total_recall, F))
        
            
def dict_of_all_score():
    """
    return score of all word in all doc"""
    DICT = {}
    for doc in QUESTION_TO_TRAINS:
            DICT[doc] = dict_doc_score(doc)
    return DICT
        
def chatting_with_corpus():

    path = 'processed/chat_tfidf.txt'
    i = 0
    questions = []
    with open(path, 'r') as f:
        for line in f:
            if i%2 ==0:
                line = line[9:]
                line = line.lower().strip()
                questions.append(line)
            i +=1
    f.close()
    questions_clean = []
    for question in questions:
        if question not in questions_clean:
            questions_clean.append(question)
    questions = random.sample(questions_clean, 50)
    for line in questions:
            LINE = clear_line(line)
            if len(line) !=0:
                index= find_right_index(LINE, DICT)
                print('Humaine: ', line)
                print('Correspond question: ', raw_questions[index])
                print('Bot: ', pairs_trains[index][1])
                print('-'*50)

                    
                    
                    

"""
Creates readable text file from SRT file.
"""
import re, sys, os

def is_time_stamp(l):
    if l[:2].isnumeric() and l[2] == ':':
        return True
    return False

def has_letters(line):
    if re.search('[a-zA-Z]', line):
        return True
    return False

def has_no_text(line):
    l = line.strip()
    if not len(l):
        return True
    if l.isnumeric():
        return True
    if is_time_stamp(l):
        return True
    if l[0] == '(' and l[-1] == ')':
        return True
    if not has_letters(line):
        return True
    return False

def is_lowercase_letter_or_comma(letter):
    if letter.isalpha() and letter.lower() == letter:
        return True
    if letter == ',':
        return True
    return False

def clean_up(lines):
    """
    Get rid of all non-text lines and
    try to combine text broken into multiple lines
    """
    new_lines = []
    for line in lines[1:]:
        if has_no_text(line):
            continue
        elif len(new_lines) and is_lowercase_letter_or_comma(line[0]):
      #combine with previous line
            line = re.sub('<[^>]+>', '', line)
            line  = re.sub('\x9c', 'oe', t )
            new_lines[-1] = new_lines[-1].strip() + ' ' + line
        else:
      #append line
            line = re.sub('<[^>]+>', '', line)
            new_lines.append(line)
    return new_lines

def main():
    """
    args[1]: file name
    args[2]: encoding. Default: utf-8.
      - If you get a lot of [?]s replacing characters,
      - you probably need to change file_encoding to 'cp1252'
    """
    for root, dirs, files in os.walk("soustitre"):
        for file in files:
            if file.endswith(".srt"):
                path = os.path.join(root, file)
                with open(path, encoding= 'iso-8859-1', errors='replace') as f:
                    lines = f.readlines()
                    new_lines = clean_up(lines)
                f.close()
                new_file_name = file[:-4] + '.txt'
                directory = 'soustitre'
                path = os.path.join('soustitre', new_file_name)
                with open(path,  'w') as f:
                    for line in new_lines:
                        f.write(line)
                f.close()
    
    
def get_attention(question):
    question_ = clear_line(question)
    index = find_right_index(question_, DICT)
    question__ = QUESTION_TO_TRAINS[index]
    print(question)
    print(DICT['normal'][question_])
    
def save_convos_in_texte(all_convos):
    
    index_of_train = random.sample(range(len(all_convos)), int(len(all_convos)*0.80))
    with open('convos3septembre_train.txt', 'w') as f:
        for index in index_of_train:
            pair = all_convos[index]
            f.write('YOU ++++ '+ pair[0]+'\n')
            f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    with open('convos3septembre_test.txt', 'w') as f:
        for index in range(len(all_convos)):
            if index not in index_of_train:
                pair = all_convos[index]
                f.write('YOU ++++ '+ pair[0]+'\n')
                f.write('BOT ++++ '+ pair[1]+'\n')
    f.close()
    len_test = len(all_convos)- len(index_of_train)
    print('save {} convos in train and {} convos in test'.format(len(index_of_train), len_test))

def minutes(seconds):
    minute = seconds//60
    seconds_ = int(seconds-minute*60)
    return '{} minutes  et {} seconds'.format(minute, seconds_)
    
def get_all_convos():
    convos = []
    raw_questions = []
    raw_convos = []
    for root, dirs, files in os.walk("soustitre"):
        for file in files:
            if file.endswith(".txt"):
                path = os.path.join(root, file)
                i = 0
                with open(path, 'r') as f:
                    get_ = False
                    for line in f.readlines():
                        line = line.strip('-')
                        if i > 3:
                            if '?' in line and get_ == False:
                                question = line.strip()
                                get_ = True
                            else:
                                if get_ == True:
                                    answer = line.strip()
                                    convos.append([clear_line(question), answer])
                                    raw_convos.append([question, answer])
                                    raw_questions.append(question)
                                    get_ = False
                          
                        i +=1
    index_of_train = random.sample(range(len(convos)), int(.8*len(convos)))
    raw_questions = [raw_questions[i] for i in index_of_train]
    pairs_trains = [convos[i] for i in index_of_train]
    pairs_tests = [convos[i] for i in range(len(convos)) if i not in index_of_train]
    return pairs_trains, pairs_tests, raw_questions, raw_convos
   
    
try:
    start
except NameError:
    start = time.time()
    pairs_trains, pairs_tests, raw_questions, raw_convos =get_all_convos()
    QUESTION_TO_TRAINS = []
    for pairs in pairs_trains:
            QUESTION_TO_TRAINS.append(pairs[0])
    DICT = dict_of_all_score()    
    try:
        pairs_trains
    except NameError :  
        print('on train:')
        test_with_precision_recall(DICT,  train = True)
        print('On test:')
        test_with_precision_recall(DICT, train = False)
        print(minutes(time.time()-start))

In [9]:
def minutes(seconds):
    minute = seconds//60
    seconds_ = int(seconds-minute*60)
    return '{} minutes  et {} seconds'.format(minute, seconds_)
print(minutes(76))

1 minutes  et 16 seconds


In [118]:
save_convos_in_texte(raw_convos)

save 2034 convos in train and 509 convos in test


In [119]:
len(raw_convos)

2543

In [115]:
chatting_with_corpus()

Humaine:  avoir combien de voiture qui roule à paris
Correspond question:  Avoir des chiens, par exemple ?
Bot:  Oh ben...
--------------------------------------------------
Humaine:  au fait, toutes mes condoléances.
Correspond question:  J'ai fait quoi au ciel ?
Bot:  Je suis pas responsable de vos soucis de couple.
--------------------------------------------------
Humaine:  pas mal.
Correspond question:  Pas de mal. Alors ?
Bot:  Vous n'êtes pas seul ?
--------------------------------------------------
Humaine:  pas du tout, sahib.
Correspond question:  Retire-t-on l'épée du rocher ou pas ?
Bot:  Il y a le pour et le contre.
--------------------------------------------------
Humaine:  a voiture
Correspond question:  Qu'y a-t-il ?
Bot:  Attila !
--------------------------------------------------
Humaine:  je veux savoir
Correspond question:  Je suis seul à savoir ?
Bot:  Les gens savent que je prends un espion, mais pas qui c'est.
--------------------------------------------------
H

In [79]:
index = random.choice(range(len(raw_convos)-10))
for i in range(20):
    print(raw_convos[i+index][1])

Derrière, vous jetterez un sort et voilà.
Vous manquez pas de souffle.
Parce que vous voulez aussi faire la menuiserie ?
Allez-y !
Faites tout.
Je vous préviens.
Je veux bien vous donner la fabrication, mais c'est mon projet.
Je ne fais rien du tout.
Si. S'il vous plaît.
À 2, c'est plus sympa.
Qu'y a-t-il ?
Rien, rien.
Vous n'avez rien à foutre dans ma tente.
Je cherche un truc.
Quoi ?
Vos plaques.
Quelles plaques ?
De dissimulation.
Vous avez pu les faire ?
Oui. Avec Elias.


In [116]:
chat_with_memory()

Bonjour, C'est le bot d'Avicen, comment ça va?
Vous: salut
bot:  Ah oui ?
Vous: comment ça va
bot:  Travailler ensemble ?
Vous: comment tu va
bot:  Travailler ensemble ?
Vous: comment vous allez
bot:  J'ai pas le droit d'essayer ?
Vous: comment allez vous?
bot:  J'ai pas le droit d'essayer ?
Vous: tout va bien?
bot:  Non, mais ils y sont pour rien.
Vous: je veux un soutien
bot:  Ça va, vous avez compris.
Vous: où est tu
bot:  Là.
Vous: comment tu t'appelles
bot:  Le boulgour.
Vous: c'est quoi ton prénom
bot:  C'était du lait chaud.
Vous: quel est ton prénom
bot:  L'amour.
Vous: quelle heure
bot:  C'est midi passé.
Vous: quand
bot:  Dans quelques jours.
Vous: quoi
bot:  Il est près de son père.
Vous: comment
bot:  Je suis toujours roi.
bot:  Tout va bien, quoi ?
Vous: qui est le roi
bot:  Cette épée-là.
Vous: quel roi
bot:  Il est chez lui, alors il va pas se gêner pour visiter un camp qu'il a payé.
bot:  Tout va bien, quoi ?
Vous: qui est le patron
bot:  Ceux-là.
Vous: qui paie
bot:  L

In [10]:
for i in range(10):
    question = random.choice(pairs_tests)
    print(question)

['je par mégard don le moindr sign de vouloir parl', 'Euh...']
['est plus la mêm', 'Excusez-moi.']
['mais parc que', 'Vous pouvez pas dormir là.']
['ça chang et ça désacralis le apen', 'Faites donc péter le butin.']
['ici ce est plus kaamelott', "C'est la zone de l'équipe à Perceval et Karadoc."]
['oh la blagu pourr', "Qu'est-ce que je fais ?"]
['est quoi quand il dit machin gnagn', "Le respect à la mémoire d'Uther le dragon ?"]
['donc yvain vous est plus chevali au lion', "Non. C'est moi, maintenant."]
['est la tradit de chez épous', 'Donc vous persistez.']
['ça veut dir quoi', "Qu'il y en a vraiment peu qui survivent."]


In [11]:
chat()

Bonjour, C'est le bot d'Avicen, pose tes questions, s'il te plaît!
Vous: hello
bot:  Vous vous foutez de nous ?
Vous: non
bot:  Mais il faut chercher quand même.
Vous: cherche quoi
bot:  Qu'est-ce que vous voulez ?
Vous: je veux juste parler
bot:  Vous prévenez trop tard.
Vous: je cherche à comprendre
bot:  J'en ai marre.
Vous: mais c'est important
bot:  Évidemment. Ne l'écoutez pas.
Vous: c'est bien noté
bot:  Bon. C'est prêt quand ?
Vous: tout de suite
bot:  C'en est pas un ?
Vous: non
bot:  Non ?
Vous: non, plusieur
bot:  Fini de glander !
Vous: d'accord
bot:  A-t-on déjà été tous d'accord ?
Vous: good, je veux bien commencer le boulot
bot:  Plus rien ne tient debout.
Vous: d'accord
bot:  Alors là, pareil : c'est banal, mais ne jamais négliger les bienfaits du couple, surtout quand on est a un poste de décision. J'ai toujours mené mes troupes de façon laborieuse.
Vous: cool
bot:  Et si l'absence de réponses était due à l'absence de Caius ?
Vous: bon, on finit ça
bot:  J'en sais rien

In [93]:
def get_all_convos_dingue():
    convos = []
    raw_questions = []
    raw_convos = []
    get_ = False
    for root, dirs, files in os.walk("soustitre"):
        for file in files:
            if file.endswith(".txt"):
                path = os.path.join(root, file)
                i = 0
                with open(path, 'r') as f:
                    for line in f.readlines():
                        line = line.strip('-').strip()
                        if i > 3:
                            if '?' in line and get_ == False:
                                question = line
                                get_  = True
                            else:
                                if get_ == True:
                                    answer = line
                                    raw_convos.append([question, answer])
                                    get_ = False
                                else:
                                    get_ = False
                                
                        i +=1
    return   raw_convos
raw_convos = get_all_convos_dingue()

In [94]:
len(raw_convos)
index = random.choice(range(len(raw_convos)-30))
for i in range(30):
    print(raw_convos[index+i])

['Je ne pose pas les bonnes questions ?', 'Soupir.']
["Comme l'autre fois, vous allez rester près du feu à me regarder et repartir comme ça ?", "C'est pas dur de dire son nom, bon Dieu."]
["J'ai pas raison ?", 'Que voulez-vous ? Ils doivent voir le roi.']
['Ils ne sont pas chevaliers, vos poulets ?', 'Mes "poulets"... Ce sont des chevaliers auxquels vous devez respect.']
['Même avec le coup des loups ?', '"Le coup des loups" ?']
["C'est quoi, là ?", "Mais c'est un assassinat."]
['Il y a aussi des bandits ?', "C'est pas possible."]
['Ah ! Vous avez entendu ?', 'Non.']
['Pourquoi sursautez-vous ?', "Vous m'avez fait sursauter."]
['Ça va pas, de dire ça ?', "C'est vrai."]
['Un grognement de loup ?', 'Un grognement de bandit ?']
['Ça fait comment ?', "On s'en fout."]
["Venec, c'est vous ?", 'On a eu peur.']
['Que faites-vous ici ?', 'En affaire. Et vous, seuls ?']
['Vous allez nous attaquer ?', 'Ou nous frapper ?']
['Vous avez du pognon ?', 'Beaucoup.']
['Ah ouais ?', "On va pouvoir s'arra

In [95]:
len(raw_convos)

2543